In [1]:
import os
import glob
import pandas as pd

In [2]:
os.chdir("/Users/3647507/OneDrive - Solistica/00 Personal/0000 BOOTCAMP/Homeworks/tableau-challenge/Data")

In [3]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames

['201901-citibike-tripdata.csv',
 '201902-citibike-tripdata.csv',
 '201903-citibike-tripdata.csv',
 '201904-citibike-tripdata.csv',
 '201905-citibike-tripdata.csv',
 '201906-citibike-tripdata.csv',
 '201907-citibike-tripdata.csv',
 '201908-citibike-tripdata.csv',
 '201909-citibike-tripdata.csv',
 '201910-citibike-tripdata.csv',
 '201911-citibike-tripdata.csv',
 '201912-citibike-tripdata.csv']

In [4]:
# Combine all files in the list
combined_df = pd.concat([pd.read_csv(f) for f in all_filenames ])

In [5]:
# Show head
combined_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,320,2019-01-01 00:01:47.4010,2019-01-01 00:07:07.5810,3160.0,Central Park West & W 76 St,40.778968,-73.973747,3283.0,W 89 St & Columbus Ave,40.788221,-73.970416,15839,Subscriber,1971,1
1,316,2019-01-01 00:04:43.7360,2019-01-01 00:10:00.6080,519.0,Pershing Square North,40.751873,-73.977706,518.0,E 39 St & 2 Ave,40.747804,-73.973442,32723,Subscriber,1964,1
2,591,2019-01-01 00:06:03.9970,2019-01-01 00:15:55.4380,3171.0,Amsterdam Ave & W 82 St,40.785247,-73.976673,3154.0,E 77 St & 3 Ave,40.773142,-73.958562,27451,Subscriber,1987,1
3,2719,2019-01-01 00:07:03.5450,2019-01-01 00:52:22.6500,504.0,1 Ave & E 16 St,40.732219,-73.981656,3709.0,W 15 St & 6 Ave,40.738046,-73.996430,21579,Subscriber,1990,1
4,303,2019-01-01 00:07:35.9450,2019-01-01 00:12:39.5020,229.0,Great Jones St,40.727434,-73.993790,503.0,E 20 St & Park Ave,40.738274,-73.987520,35379,Subscriber,1979,1


In [6]:
# Show the size of the dataframe
combined_df.shape

(20551697, 15)

In [7]:
# Show data types
combined_df.dtypes

tripduration                 int64
starttime                   object
stoptime                    object
start station id           float64
start station name          object
start station latitude     float64
start station longitude    float64
end station id             float64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
birth year                   int64
gender                       int64
dtype: object

In [8]:
# Split string date column to take out decimals
new_datetime = combined_df['starttime'].str.split(".", n = 1, expand = True)

# Set the columns value
combined_df['starttime'] = new_datetime[0]

# Convert to Datetime
combined_df['starttime'] = pd.to_datetime(combined_df['starttime'])

In [9]:
# Calculate the week number (from 1 to 52)
combined_df['weeknum'] = combined_df['starttime'].dt.isocalendar().week

In [10]:
# Create a summary dataframe  by week number and some other important fields
weeknum_summary = combined_df.groupby(['start station id', 'start station name', 'start station latitude', 'start station longitude', 'usertype', 'gender', 'weeknum']).agg(
    total_trips = ('bikeid', 'count'),
    avg_trip_duration = ('tripduration', 'mean'),
    med_trip_duration = ('tripduration', 'median')  
    
)

In [11]:
# Reset Indexes
weeknum_summary.reset_index(inplace=True)

In [12]:
# Convert to CSV
weeknum_summary.to_csv('Output/weeknum_summary.csv')

In [13]:
# Create a summary dataframe from to stations
fromto_summary = combined_df.groupby(['start station id', 'start station name', 'start station latitude', 'start station longitude', 
                                      'end station id', 'end station name', 'end station latitude', 'end station longitude']).agg(
    total_trips = ('bikeid', 'count'),
    avg_trip_duration = ('tripduration', 'mean'),
    med_trip_duration = ('tripduration', 'median')  
    
)

In [14]:
# Reset Indexes
fromto_summary.reset_index(inplace=True)

In [15]:
# Convert to CSV
fromto_summary.to_csv('Output/from_to_stations.csv')